<a href="https://colab.research.google.com/github/swapnaGyabgonda/Video-Recommendation-System/blob/main/videorecomndationsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Upload movie dataset
df = pd.read_csv('/content/moviedata.csv')
df.head()

,title,transcript,Preprocessed
0,astronomy,astronomy literally meaning the science that s...,astronomy literally mean science study law sta...
1,white dwarf,a white dwarf also called a degenerate dwarf i...,white dwarf also call degenerate dwarf stellar...
2,chimp,chimpanzees are great apes found across centra...,chimpanzee great ape find across central west ...
3,crypto,what are the top four crypto exchanges india w...,top four crypto exchange india buy two poker c...
4,crypto_tax,india recently introduced crypto tax in the bu...,india recently introduce crypto tax budget two...


In [ ]:
df.columns

Index(['title', 'transcript', 'Preprocessed'], dtype='object')

In [ ]:
df.isnull().sum()

,0
title,0
transcript,0
Preprocessed,0


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['Preprocessed'] = df['Preprocessed'].str.lower()

In [ ]:
import re
df['Preprocessed'] = df['Preprocessed'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['Preprocessed'] = df['Preprocessed'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Preprocessed'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def get_recommendations(title, df, cosine_sim):
    idx = df[df['title'] == title].index[0]  # Get index of the movie
    sim_scores = list(enumerate(cosine_sim[idx]))  # Similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Sort by score
    sim_scores = sim_scores[1:6]  # Top 5 recommendations
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

In [ ]:
print(df['title'].unique())

['astronomy' 'white dwarf' 'chimp' 'crypto' 'crypto_tax' 'crypto_means'
 'digital_token' 'cryptocurrency' 'dodo' 'dolphin' 'elephant'
 'fd_mutual_finance' 'finance' 'jeffrey_archer' 'mammoth' 'murakami'
 'mutual_funds' 'paulo_coelho' 'penguin' 'animal_farm' 'shakespeare'
 'snow_bunting' 'super_nova' 'vaccine_news' 'whale']


In [ ]:
get_recommendations('dodo', df, cosine_sim)

,title
18,penguin
14,mammoth
2,chimp
9,dolphin
10,elephant


In [ ]:
!pip install Flask

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import pickle

app = Flask(__name__)

# Load your preprocessed data and model
df = pd.read_csv('/content/moviedata.csv')
cosine_sim = pickle.load(open('cosine_similarity.pkl', 'rb'))

def get_recommendations(title, df, cosine_sim):
    idx = df[df['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices].tolist()

@app.route('/recommend', methods=['GET'])
def recommend():
    title = request.args.get('title')
    if title not in df['title'].values:
        return jsonify({'error': 'Title not found'})

    recommendations = get_recommendations(title, df, cosine_sim)
    return jsonify({'Recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'cosine_similarity.pkl'

In [ ]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Apply TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Preprocessed'])

# Calculate Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Save model using pickle
pickle.dump(cosine_sim, open('cosine_similarity.pkl', 'wb'))

In [ ]:
cosine_sim = pickle.load(open('/content/cosine_similarity.pkl', 'rb'))

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import pickle

app = Flask(__name__)

# Load data and model
df = pd.read_csv('/content/moviedata.csv')
cosine_sim = pickle.load(open('/content/cosine_similarity.pkl', 'rb'))

@app.route('/recommend', methods=['GET'])
def recommend():
    title = request.args.get('title')
    if title not in df['title'].values:
        return jsonify({'error': 'Title not found'})

    recommendations = get_recommendations(title, df, cosine_sim)
    return jsonify({'Recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

In [ ]:
# Import Libraries
from flask import Flask, request, jsonify
import pandas as pd
import pickle
from pyngrok import ngrok

# App Initialization
app = Flask(__name__)

# Load Model
movies = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

@app.route('/')
def home():
    return "Hello Swapna 🎯 Movie Recommendation API is Running"

@app.route('/recommend', methods=['GET'])
def recommend():
    movie = request.args.get('title')
    if movie not in movies['title'].values:
        return jsonify({"error": "Movie not found"})

    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommendations = []
    for i in distances[1:6]:
        recommendations.append(movies.iloc[i[0]].title)

    return jsonify(recommendations)

# Ngrok Tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)
app.run(debug=True, use_reloader=False)

FileNotFoundError: [Errno 2] No such file or directory: 'movies.pkl'